In [1]:
data = open("data.csv").read().strip().split("\n")
sep_data = [item.split("\t") for item in data]

In [2]:
len(sep_data)

1018

In [4]:
alen = set([len(item) for item in sep_data]) 

In [5]:
alen

{8}

In [6]:
sep_data[0]

['0',
 '小米空气净化器2多少钱',
 '您好，小米空气净化器2现在售价599元',
 '最新的空气净化器多少钱',
 '小米最新的空气净化器多少钱',
 '最新款的空气净化器多贵',
 '我想了解一下最新的空气净化器的价位',
 '现在最新的空气净化器价位是怎样的']

In [7]:
import jieba
seg_data = []

In [8]:
col_data = list(zip(*sep_data))

In [10]:
flag = [1, 3, 4, 5, 6, 7]

In [18]:
def segment_data(ch_data):
    result = []
    for item in ch_data:
        result.append(" ".join(jieba.cut(item)))
    return result

In [24]:
seg_data = []
for i, item in enumerate(col_data):
    if i in flag:
        seg_data.append(segment_data(item))
    else:
        seg_data.append(item)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/81/4j4qwk416tb_47lk0lw234f00000gn/T/jieba.cache
Loading model cost 0.726 seconds.
Prefix dict has been built succesfully.


In [26]:
len(seg_data)

8

In [27]:
out_data = list(zip(*seg_data))

In [28]:
len(out_data)

1018

In [29]:
out_data[0]

('0',
 '小米 空气 净化器 2 多少 钱',
 '您好，小米空气净化器2现在售价599元',
 '最新 的 空气 净化器 多少 钱',
 '小米 最新 的 空气 净化器 多少 钱',
 '最新款 的 空气 净化器 多贵',
 '我 想 了解 一下 最新 的 空气 净化器 的 价位',
 '现在 最新 的 空气 净化器 价位 是 怎样 的')

In [30]:
ids = seg_data[0]
questions = seg_data[1]
answers = seg_data[2]
print(ids[:2], questions[:2], answers[:2])

('0', '1') ['小米 空气 净化器 2 多少 钱', '小米 空气 净化器 2 效果 如何'] ('您好，小米空气净化器2现在售价599元', '您好，小米空气净化器2颗粒物CADR高达310m^3/h')


In [31]:
import json

In [33]:
js_data = {"ids": ids, "questions": questions, "answers": answers}
json.dump(js_data, open("qa.json", "w"))

In [34]:
open("segment_data", "w").write("\n".join(["\t".join(item) for item in out_data]))

125926

In [35]:
import random    

In [64]:
def random_neg_sample(raw_data):
    collect_data = list(zip(*raw_data))
    all_ids = set(list(range(len(collect_data) * 6)))
    print(len(all_ids))
    result = []
    for i, item in enumerate(collect_data):
        klist = list(all_ids - set(list(range(i*6, (i+1)*6))))
        if i == 0: print(len(klist))
        assert len(klist) == len(all_ids) - 6, (len(klist), len(all_ids))
        j = 0
        len_item = len(item)
        while( j < len_item - 1):
            k = len_item - 1 - j
            samples = random.sample(klist, k)
            neg_sample = []
            for kl in samples:
                loc = kl // 6 
                inloc = kl % 6
                neg_sample.append([item[j], collect_data[loc][inloc], 0])
            pos_sample = []
            for pos in item[j+1:]:
                pos_sample.append([item[j], pos, 1])
            assert len(pos_sample) == len(neg_sample), (len(pos_sample), len(neg_sample), k)
            result.extend(neg_sample)
            result.extend(pos_sample)
            j += 1
    random.shuffle(result)
    return result

In [37]:
seg_data[0][:3]

('0', '1', '2')

In [38]:
all_q = [seg_data[1], seg_data[3], seg_data[4], seg_data[5], seg_data[6], seg_data[7]]

In [39]:
all_q_t = list(zip(*all_q))
train_q_t = all_q_t[:950]
dev_q_t = all_q_t[950:]

In [40]:
train_q_t[:3]

[('小米 空气 净化器 2 多少 钱',
  '最新 的 空气 净化器 多少 钱',
  '小米 最新 的 空气 净化器 多少 钱',
  '最新款 的 空气 净化器 多贵',
  '我 想 了解 一下 最新 的 空气 净化器 的 价位',
  '现在 最新 的 空气 净化器 价位 是 怎样 的'),
 ('小米 空气 净化器 2 效果 如何',
  '新 的 空气 净化器 性能 如何',
  '小米 空气 净化器 2 的 净化 效果 如何',
  '小米 空气 净化器 2 的 净化 性能 如何',
  '新 的 空气 净化器 能力 怎样',
  '新 的 空气 净化器 能 达到 怎样 的 效果'),
 ('小米 空气 净化器 2 占地 大 吗',
  '新 的 空气 净化器 大小 有 多 大',
  '新 的 空气 净化器 体积 有 多 大',
  '小米 空气 净化器 2 体积 上 怎样',
  '小米 空气 净化器 2 有 多 大',
  '新 的 空气 净化器 不会 很 占地 吧')]

In [41]:
train_q = list(zip(*train_q_t))
eval_q = list(zip(*dev_q_t))

In [63]:
train_q[0][:2]
eval_q[0][:2]
print(len(train_q_t), len(train_q[0]))

950 950


In [65]:
train = random_neg_sample(train_q)

5700
5694


In [66]:
len(train)

28500

In [67]:
sum([item[2] for item in train])

14250

In [71]:
dev = random_neg_sample(eval_q)
print(len(dev))
print(sum([item[2] for item in dev]))

408
402
2040
1020


In [74]:
def write_data(origin_data, dtype="train"):
    json.dump(origin_data, open("origin_" + dtype + ".json", "w"))
    q1, q2, label = list(zip(*origin_data))
    ids = list(range(len(label)))
    json.dump({"ids":ids, "q1": q1, "q2": q2, "label":label}, open(dtype+".json", "w"))

In [75]:
write_data(train)
write_data(dev, "dev")